In [ ]:
#Установка нужных пакетов
!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer, WordNetLemmatizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 10.2 MB/s 
     |████████████████████████████████| 18.5 MB 644 kB/s 
     |████████████████████████████████| 88 kB 8.0 MB/s 
     |████████████████████████████████| 1.1 MB 46.2 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=689551fcb94ccc5b151b7086c703f163dc32c7a96b80b888e3bb5b7e5a11049b
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55512 sha256=5f56159cc4d78d52d54038876b84e54ae4757089c7be45acf8e30e2324b13cce
  Stored in directory: /root/.cache/pip/wheels/19/bc/eb/974072a56a7082a302f8b4be1ad6d21bf5019235c2eff65928
Successfully built umap-learn pynndescent
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling ge

ImportError: ignored

In [ ]:
# выгружаем датасет:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

--2022-11-03 15:47:43--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2022-11-03 15:47:43--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd7b3e925bb47257ecad7a5f9f2.dl.dropboxusercontent.com/cd/0/get/BwDpQONx1RcEyXvX-npoTxImcmAGvjl6mqgkF4zz5f0FEWI2OjXPLryQsWrN7xV7YQKqxRs9eji4BbgUJ6KC0h5gSpqRBFEfuVl9enOtapfT9LhKgB9W1xyQvIuI8KQ8A0zD42EBCU9GbrC8UeNTRfo1wijibkRK0Yjh7wy6PYRpbA/file?dl=1# [following]
--2022-11-03 15:47:43--  https://ucd7b3e925bb47257ecad7a5f9f2.dl.dropboxusercontent.com/cd/0/get/BwDpQONx1RcEyXvX-npoTxImcmAGvjl6mqgkF4zz5f0FEWI2OjXPLryQsWrN7xV7YQKqxRs9eji4BbgUJ6KC0h5

In [ ]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [ ]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


#Задание 1: Перевести все слова в нижний регистр (NLTK) из data и добавьте как лист токенов в листе data_tok


In [ ]:
data_tok = []
for i in range(len(data)): 
  data_tok.append(tokenizer.tokenize(data[i].lower()))
  
#checking
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"


###Задание 2: Подсчитайте топ10 самых популярных лем в рамках data

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

dict_words = {}

for token in data_tok:
  for word in token: 
    word_lemmatizing = lemmatizer.lemmatize(word)
    if word_lemmatizing in dict_words.keys(): 
      dict_words[word_lemmatizing] += 1
    else:
      dict_words[word_lemmatizing] = 1

import pandas as pd
df = pd.DataFrame.from_dict(dict_words, orient='index')
df
df.sort_values(0, ascending = False).head(10)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,0
?,552413
the,252068
what,214798
is,185392
a,172513
i,149873
to,141788
in,139813
how,135687
of,112001


###Задание 3: Подсчитайте количество разных слов до и после лемматизации

In [ ]:
tok_words = {}
for token in data_tok:
  for word in token: 
    if word in tok_words.keys(): 
      tok_words[word] += 1
    else:
      tok_words[word] = 1

In [ ]:
#raw word count
word_size = len(tok_words.values())
#size of lemmatized words
lem_size = len(dict_words.values())
print ( f' Raw words count : {word_size} \n Lemmatized words count : {lem_size}')

 Raw words count : 87819 
 Lemmatized words count : 80303


###Задание 4: Подсчитайте количество разных слов до и после стемминга

In [ ]:
from nltk.stem.porter import *

stemmer = PorterStemmer()

In [ ]:
stem_words = {}
for token in data_tok:
  for word in token: 
    stem_word = stemmer.stem(word)
    if stem_word in stem_words.keys(): 
      stem_words[stem_word] += 1
    else:
      stem_words[stem_word] = 1

In [ ]:
#stem word count
stem_size = len(stem_words.values())
#size of lemmatized words
lem_size = len(dict_words.values())
print ( f' Stemmed words count : {stem_size} \n Lemmatized words count : {lem_size}')

 Stemmed words count : 67026 
 Lemmatized words count : 80303


###Задание 5: Подсчитайте количество разных слов



REGEXP

https://www.programiz.com/python-programming/regex 

https://docs.python.org/3/howto/regex.html

In [ ]:
import re

pattern = 'a[a-zA-Z]*s'
test_string = 'abyss'
result = re.match(pattern, test_string)

if result:
  print("Search successful.")
else:
  print("Search unsuccessful.")	

Search successful.


###Задание 6: 
https://www.hackerrank.com/challenges/matching-specific-string/problem?isFullScreen=true 

###Задание 7: 
https://www.hackerrank.com/challenges/matching-whitespace-non-whitespace-character/problem?isFullScreen=true

###Задание 8: 
https://www.hackerrank.com/challenges/matching-start-end/problem?isFullScreen=true

###Задание 9: 
https://www.hackerrank.com/challenges/matching-word-boundaries/problem?isFullScreen=true

Bag Of Words (BOW)

In [ ]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input 
    and returns a vector of size of filtered_vocab.It puts 0 if the 
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector
def unique(sequence):
    '''This functions returns a list in which the order remains 
    same and no item repeats.Using the set() function does not 
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#create a list of stopwords.You can import stopwords from nltk too
stopwords=["to","is","a"]

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]

#Write the sentences in the corpus,in our case, just two 
string1="Welcome to Great Learning , Now start learning"
string2="Learning is a good practice"

#convert them to lower case
string1=string1.lower()
string2=string2.lower()

#split the sentences into tokens
tokens1=string1.split()
tokens2=string2.split()
print(tokens1)
print(tokens2)

#create a vocabulary list
vocab=unique(tokens1+tokens2)
print(vocab)

#filter the vocabulary list
filtered_vocab=[]
for w in vocab: 
    if w not in stopwords and w not in special_char: 
        filtered_vocab.append(w)
print(filtered_vocab)

#convert sentences into vectords
vector1=vectorize(tokens1)
print(vector1)
vector2=vectorize(tokens2)
print(vector2)

['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'learning']
['learning', 'is', 'a', 'good', 'practice']
['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'is', 'a', 'good', 'practice']
['welcome', 'great', 'learning', 'now', 'start', 'good', 'practice']
[1, 1, 2, 1, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 1]


Задание 10: Реализовать Bag of words на data_tok (можно на NLTK, можно без)